## Guided Latent Dirichlet Allocation

### Import data

In [1]:
import json

with open('preprocessing/data.json', 'r') as f:
    documents = json.load(f)
    texts = [' '.join([word for word in text]) for text in documents]

### Create dictionary and corpus

In [2]:
from gensim import corpora

# Map each token to a unique ID
dictionary = corpora.Dictionary(documents)
print(f'Number of unique tokens: {len(dictionary)}')

# Filter out tokens by frequency
min_doc, max_doc = 15, .3
dictionary.filter_extremes(no_below=min_doc, no_above=max_doc)
print(f'Number of unique tokens (filtered): {len(dictionary)}')

Number of unique tokens: 43392
Number of unique tokens (filtered): 5034


### Store dictionary items as words in vocabulary

In [3]:
tmp = []

for i in range(len(dictionary)):
    tmp.append(dictionary.get(i))
vocab = tuple(tmp)

word2id = dict((v, idx) for idx, v in enumerate(vocab))

### Create a sparse matrix from gensim's BOW document representation

In [4]:
import numpy as np
from gensim import matutils

def bow_iterator(docs, dictionary):
    for doc in docs:
        yield dictionary.doc2bow(doc)

def get_term_matrix(msgs, dictionary):
    bow = bow_iterator(msgs, dictionary)
    X = np.transpose(matutils.corpus2csc(bow).astype(np.int64))
    return X


X = get_term_matrix(documents, dictionary)

### Run the Guided LDA model w/o seed topics

In [5]:
from lda import guidedlda as glda

n_topics = 15

# Train the model
glda_model = glda.GuidedLDA(alpha=.1, n_topics=n_topics, n_iter=300, \
                            random_state=7, refresh=20)

# Run the model
glda_model.fit(X, seed_confidence=0.6)

INFO:lda:n_documents: 2289
INFO:lda:vocab_size: 5034
INFO:lda:n_words: 901355
INFO:lda:n_topics: 15
INFO:lda:n_iter: 300
INFO:lda:<0> log likelihood: -9869598
INFO:lda:<20> log likelihood: -7025737
INFO:lda:<40> log likelihood: -6903803
INFO:lda:<60> log likelihood: -6857930
INFO:lda:<80> log likelihood: -6831842
INFO:lda:<100> log likelihood: -6822835
INFO:lda:<120> log likelihood: -6815988
INFO:lda:<140> log likelihood: -6809858
INFO:lda:<160> log likelihood: -6806122
INFO:lda:<180> log likelihood: -6802483
INFO:lda:<200> log likelihood: -6799893
INFO:lda:<220> log likelihood: -6798088
INFO:lda:<240> log likelihood: -6796804
INFO:lda:<260> log likelihood: -6796488
INFO:lda:<280> log likelihood: -6792839
INFO:lda:<299> log likelihood: -6792371


### Print the results

In [6]:
n_top_words = 15
topic_word = glda_model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print(f"Topic {i}: {' '.join(topic_words)}\n")

Topic 0: бактерия белок днк геном вирус последовательность эукариот гриб фермент белка клеточный штамм бактериальный митохондрия паразит

Topic 1: галактика звезда масса чёрный_дыра излучение вселенная планета наблюдение космический солнечный солнце гравитационный телескоп объект спектр

Topic 2: рыба динозавр млекопитающее кость находка остаток ископаемое палеонтолог зуб древний хищник морской представитель скелет строение

Topic 3: белок мышь опухоль ткань экспрессия рак рецептор заболевание кровь днк клеточный активность нормальный иммунный раковый

Topic 4: эволюционный предок нервный позвоночный червь многоклеточный личинка стадия ветвь губка экспрессия билатерия происхождение эмбрион древний

Topic 5: мутация популяция отбор генетический геном эволюционный хромосома линия вариант аллель приспособленность полезный поколение половой частота

Topic 6: поле волна квантовый теория электрон атом свет скорость температура движение сила магнитный_поле энергия излучение частота

Topic 7: 

### Run the model with seed topics

In [7]:
seed_topic_list = [
    ['сапиенс', 'пещера', 'неандерталец', 'находка', 'кость'],
    ['галактика', 'чёрный_дыра', 'вселенная', 'космический'],
    ['опухоль', 'рак', 'метастаз', 'раковый', 'мутация'],
    ['коллайдер', 'распад', 'детектор', 'бозон_хиггс', 'кварк'],
    ['атмосфера', 'содержание', 'климат', 'концентрация'],
    ['нейрон', 'мозг', 'рецептор', 'сигнал', 'нервный']
]

In [8]:
seed_topics = {}

for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

# Run the model
glda_model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

INFO:lda:n_documents: 2289
INFO:lda:vocab_size: 5034
INFO:lda:n_words: 901355
INFO:lda:n_topics: 15
INFO:lda:n_iter: 300
INFO:lda:<0> log likelihood: -9866300
INFO:lda:<20> log likelihood: -6948117
INFO:lda:<40> log likelihood: -6871008
INFO:lda:<60> log likelihood: -6845928
INFO:lda:<80> log likelihood: -6831028
INFO:lda:<100> log likelihood: -6825812
INFO:lda:<120> log likelihood: -6822201
INFO:lda:<140> log likelihood: -6819377
INFO:lda:<160> log likelihood: -6817914
INFO:lda:<180> log likelihood: -6813547
INFO:lda:<200> log likelihood: -6807150
INFO:lda:<220> log likelihood: -6804433
INFO:lda:<240> log likelihood: -6801661
INFO:lda:<260> log likelihood: -6799339
INFO:lda:<280> log likelihood: -6798027
INFO:lda:<299> log likelihood: -6799206


### Print the results

In [9]:
topic_word = glda_model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print(f"Topic {i}: {' '.join(topic_words)}\n")

Topic 0: неандерталец древний геном сапиенс предок орудие шимпанзе кость человеческий возраст культура днк пещера популяция неандертальский

Topic 1: галактика звезда масса излучение чёрный_дыра вселенная волна гравитационный космический наблюдение скорость солнечный планета объект солнце

Topic 2: белок днк мышь опухоль вирус ткань белка рак экспрессия заболевание клеточный мутация иммунный молекула рецептор

Topic 3: частица энергия физика электрон квантовый теория нейтрино поле измерение детектор ядро масса протон фотон распад

Topic 4: вода температура вымирание морской климат рост остров район лес численность содержание сообщество территория скорость океан

Topic 5: мозг нейрон сигнал поведение мышь нервный рецептор активность кора память глаз информация способность стимул действие

Topic 6: бактерия геном эукариот белок днк последовательность фермент гриб водоросль митохондрия бактериальный архея штамм клеточный мембрана

Topic 7: популяция мутация отбор генетический геном эволюц